# Intro

This notebook contains the implementation of Hermite elements in FIAT as well as multiple tests. See:

* [hermite.tm](hermite.tm) for the math involved. 
* [interpolation.ipynb](interpolation.ipynb) for the implementation of nodal interpolation of Hermite (PointDerivative) dofs.
* [quadrature.ipynb](quadrature.ipynb) for the implementation of the quadrature representation of forms requiring Hermite transforms.
* [boundary.ipynb](boundary.ipynb) for details and problems related to the specification of *essential* boundary conditions.

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

# Dual set
This is the final implementation, as it is now inside the FIAT code:

In [ ]:
from FIAT import finite_element, polynomial_set, dual_set, functional, reference_element

class CubicHermiteDualSet(dual_set.DualSet):
    """The dual basis for Cubic Hermite elements.
    This class works for simplices of any dimension.
    Nodes are point evaluation and derivatives at vertices,
    plus point evaluation at the barycenter of each face if
    the spatial dimension is >= 2. """

    def __init__(self, ref_el):
        entity_ids = {}
        nodes = []
        cur = 0

        top = ref_el.get_topology()
        verts = ref_el.get_vertices()
        sd = ref_el.get_spatial_dimension()

        # Vertex dofs: one point evaluation and sd partial derivatives
        entity_ids[0] = {}
        for v in sorted(top[0]):
            nodes.append(functional.PointEvaluation(ref_el, verts[v]))
            pd = functional.PointDerivative
            for i in range(sd):
                alpha = [0] * sd
                alpha[i] = 1

                nodes.append(pd(ref_el, verts[v], alpha))

            entity_ids[0][v] = list(range(cur, cur + 1 + sd))
            cur += sd + 1

        # no edge dofs
        entity_ids[1] = {}
        for e in sorted(top[1]):
            entity_ids[1][e] = []

        if sd >= 2:
            # face dof
            # point evaluation at barycenter
            entity_ids[2] = {}
            for f in sorted(top[2]):
                pt = ref_el.make_points(2, f, 3)[0]
                n = functional.PointEvaluation(ref_el, pt)
                nodes.append(n)
                entity_ids[2][f] = list(range(cur, cur + 1))
                cur += 1

            # No more dof in higher dimensions
            for dim in range(3, sd + 1):
                entity_ids[dim] = {x: [] for x in sorted(top[dim])}

        super(CubicHermiteDualSet, self).__init__(nodes, ref_el, entity_ids)

# Finite element

In [ ]:
class CubicHermite(finite_element.CiarletElement):
    """ The cubic Hermite finite element.
    There are three degrees of freedom per vertex and one additional
    for each barycenter of a 2d face. """

    def __init__(self, ref_el, degree=3):
        # Degree is fixed to 3 for cubic Hermite elements.
        poly_set = polynomial_set.ONPolynomialSet(ref_el, 3)
        dual = CubicHermiteDualSet(ref_el)
        super(CubicHermite, self).__init__(poly_set, dual, 3)

    def mapping(self):
        # MBD FIXME: is this correct?
        mappings = []
        for f in self.dual_basis():
            if isinstance(f, functional.PointEvaluation):
                mappings.append("affine")
            elif isinstance(f, functional.PointDerivative):
                mappings.append("hermite")
        return mappings


# Tests

## Construction of polynomial set

This is taken from the constructor of `FIAT.finite_element.FiniteElement()`.

In [ ]:
# build generalized Vandermonde matrix
K = reference_element.ReferenceElement(shape = reference_element.LINE,
                                       vertices = ((-1.0,),(1.0,)),
                                       topology = reference_element.UFCInterval().get_topology())
poly_set = polynomial_set.ONPolynomialSet(K, 3)
old_coeffs = poly_set.get_coeffs()
dual = CubicHermiteDualSet(K)
dualmat = dual.to_riesz(poly_set)

shp = dualmat.shape
if len(shp) > 2:
    num_cols = np.prod(shp[1:])

    A = np.reshape(dualmat, (dualmat.shape[0], num_cols))
    B = np.reshape(old_coeffs, (old_coeffs.shape[0], num_cols))
else:
    A = dualmat
    B = old_coeffs

V = np.dot(A, np.transpose(B))

Vinv = np.linalg.inv(V)

new_coeffs_flat = np.dot(np.transpose(Vinv), B)

new_shp = tuple([new_coeffs_flat.shape[0]] + list(shp[1:]))
new_coeffs = np.reshape(new_coeffs_flat, new_shp)

#poly_set = PolynomialSet(K, poly_set.get_degree(), poly_set.get_embedded_degree(),
#                         poly_set.get_expansion_set(), new_coeffs, poly_set.get_dmats())

In [ ]:
H = CubicHermite(K)
print(np.round(H.poly_set.tabulate_new(K.get_vertices()),2))
#print(np.round(H.poly_set.tabulate(K.get_vertices())[(0,0)],2))

## Transformation of basis functions

This is copied from `transform_hermite.py`, in a previous release of FIAT. It tests whether the nodal basis evaluated on a lattice over the reference element and over a transformed  element transforms well under an affine mapping.

Basically, what we do is we compute the "hermite" transformation in `evaluatebasis.py` by hand:

1. Tabulate (compute) the values of each basis function of the reference element over a lattice of points in the triangle, then multiply it by an adequate transformation matrix (mixing the values for the nodal basis functions related to derivatives).
2. Tabulate the values of each basis function defined over a different simplex.
3. Compare both results

In [ ]:
# Let's set up the reference triangle and another one
Khat = reference_element.UFCTriangle()
K = reference_element.ReferenceElement(shape = reference_element.TRIANGLE,
                                       vertices = ((-1.0,-1.0),(1.0,-1.0),(-1.0,1.0)),
                                       topology = Khat.get_topology())

# Construct the affine mapping between them
A, b = reference_element.make_affine_mapping(K.get_vertices(), Khat.get_vertices())

# build the Hermite element on the two triangles
Hhat = CubicHermite(Khat)
H = CubicHermite(K)

# get some points on each triangle
pts_hat = Khat.make_lattice(6)
pts = K.make_lattice(6)

# as a sanity check on the affine mapping, make sure pts map to pts_hat
for i, p in enumerate(pts):
    if not np.allclose(pts_hat[i], np.dot(A, p) + b):
        print("barf")

# Tabulate the Hermite basis on each triangle
# Each column is the value of one nodal basis function evaluated at every point
Hhat_tabulated = Hhat.get_nodal_basis().tabulate_new(pts_hat)
H_tabulated = H.get_nodal_basis().tabulate_new(pts)

### Prepare global transformation matrix
M = np.zeros((10,10), dtype=np.double)
Ainv = np.linalg.inv(A)

# Point values are transformed as is
# Derivative values are "mixed" with the Jacobian (see above)
M[0,0] = 1.0
M[3,3] = 1.0
M[6,6] = 1.0
M[9,9] = 1.0
M[1:3,1:3] = np.transpose(Ainv)
M[4:6,4:6] = np.transpose(Ainv)
M[7:9,7:9] = np.transpose(Ainv)

print(np.allclose(H_tabulated, np.dot(M.T, Hhat_tabulated)))

In [ ]:
Hhat_tabulated.shape

In [ ]:
basis = H.get_nodal_basis()
np.round(basis.tabulate_new([(-1,-1),(-1,0),(0,0)]))

# Misc

## What are dmats?

Dmats are "expansion coefficients for basis function derivatives", but I need to understand better the structure. 

In [ ]:
from FIAT import lagrange
Hhat = lagrange.Lagrange(Khat, 1)
basis = Hhat.get_nodal_basis()
dmats = basis.get_dmats()
print(len(dmats), dmats[0].shape, dmats[1].shape)
print(np.round(dmats[0], 1))
print(np.round(dmats[1], 1))